# Business Case Analysis

In this business case analysis I am using an online retail store dataset found in the [Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Online+Retail). The aim of this dataset is to identify models of purchases across the time. I will perform in this dataset:
- Apply descriptive analytics to understand data behavior.
- Apply NLP to identify groups of products.
- Apply forecasting

In [1]:
import pandas as pd
from pandas import ExcelFile
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import itertools
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import sys
from pandas.plotting import autocorrelation_plot
import warnings
import statsmodels.api as sm
import seaborn as sns
from fbprophet import Prophet

ModuleNotFoundError: No module named 'fbprophet'

In [2]:
!sys{/home/none/init_venv/bin/python -m pip install prophet}

/usr/bin/sh: 1: sys{/home/none/init_venv/bin/python: not found


In [ ]:
data_original = pd.read_csv('Online Retail.csv')

In [ ]:
data_original

# Descriptive Analytics

In [ ]:
data_original['Total_Price'] = data_original['Quantity'] * data_original['UnitPrice']

In [ ]:
data_invoice = data_original[['InvoiceNo','Quantity','Total_Price']]

In [ ]:
data_original

In [ ]:
# Get attributes per Invoice
grp_invoice = data_invoice.groupby(['InvoiceNo']).sum()

In [ ]:
grp_invoice.columns = ['Quantity','Total_Price']

### Quantity analysis

In [ ]:
grp_invoice['Quantity']

In [ ]:
#Invoice with the biggest amount of articles
grp_invoice['Quantity'].max()

In [ ]:
#Invoice with the less amount of articles
grp_invoice['Quantity'].min()

In [ ]:
#Invoice quantity distribution, it seems to have a tendency of a normal distribution.
grp_invoice['Quantity'].hist()

In [ ]:
ax = sns.countplot(x="Quantity",data=grp_invoice)

In [ ]:
#standard deviation
grp_invoice['Quantity'].std()

In [ ]:
#mean
grp_invoice['Quantity'].mean()

In [ ]:
#median
grp_invoice['Quantity'].median()

In [ ]:
#q1
grp_invoice['Quantity'].quantile(0.25)

In [ ]:
#q3
grp_invoice['Quantity'].quantile(0.75)

In [ ]:
#IQR 
grp_invoice['Quantity'].quantile(0.75) - grp_invoice['Quantity'].quantile(0.25)

### Total_Price Analysis

In [ ]:
grp_invoice['Total_Price']

In [ ]:
grp_invoice['Total_Price'].min()

In [ ]:
grp_invoice['Total_Price'].max()

In [ ]:
grp_invoice['Total_Price'].hist()

In [ ]:
ax = sns.countplot(x="Total_Price",data=grp_invoice)

In [ ]:
#standard deviation
grp_invoice['Total_Price'].std()

In [ ]:
#unit price mean
grp_invoice['Total_Price'].mean()

In [ ]:
#unit price median
grp_invoice['Total_Price'].median()

In [ ]:
#q1
grp_invoice['Total_Price'].quantile(0.25)

In [ ]:
#q3
grp_invoice['Total_Price'].quantile(0.75)

In [ ]:
#iqr
grp_invoice['Total_Price'].quantile(0.75) - grp_invoice['Total_Price'].quantile(0.25)

### Customer id analysis amount of purchases

In [ ]:
customer_purchase = data_original[['CustomerID','Total_Price']]

In [ ]:
customer_purchase

In [ ]:
#count the number of purchases per customer
number_purchases_per_customer = customer_purchase['CustomerID'].value_counts()

In [ ]:
number_purchases_per_customer

In [ ]:
#Purchase with the maximum amount of money.

print(f'customer number_purchases {number_purchases_per_customer.max()}')

In [ ]:
#Purchase with the minimum amount of money.

print(f'customer number_purchases {number_purchases_per_customer.min()}')

In [ ]:
#Purchases standard deviation of purchases.
number_purchases_per_customer.std()

In [ ]:
#Purchases mean.
number_purchases_per_customer.mean()

In [ ]:
#Purchases median.
number_purchases_per_customer.median()

In [ ]:
#Purchases q1.
number_purchases_per_customer.quantile(0.25)

In [ ]:
#Purchases q3.
number_purchases_per_customer.quantile(0.75)

In [ ]:
#iqr amount of purchases
number_purchases_per_customer.quantile(0.75) - number_purchases_per_customer.quantile(0.25)

In [ ]:
#looks like a skewed distribution to the left the amount of purchases made by each customer.
number_purchases_per_customer.hist()

 ### Customer id analysis amount of money spent per purchase

In [ ]:
customer_spent = customer_purchase.groupby('CustomerID').agg(sum)

In [ ]:
customer_spent.columns = ['Total_Price']

In [ ]:
customer_spent

In [ ]:
#amount of money spent per customer distribution. It has a skewed distribution to the left.
customer_spent['Total_Price'].hist()

In [ ]:
#The highest amount of money spent per customer.
customer_spent[customer_spent['Total_Price'] == customer_spent['Total_Price'].max()]

In [ ]:
#The least amount of money spent per customer.
customer_spent[customer_spent['Total_Price'] == customer_spent['Total_Price'].min()]

In [ ]:
#the standard deviation
customer_spent['Total_Price'].std()

In [ ]:
#the mean
customer_spent['Total_Price'].mean()

In [ ]:
#the median
customer_spent['Total_Price'].median()

In [ ]:
#q1
customer_spent['Total_Price'].quantile(0.25)

In [ ]:
#q3
customer_spent['Total_Price'].quantile(0.75)

In [ ]:
#iqr
customer_spent['Total_Price'].quantile(0.75) - customer_spent['Total_Price'].quantile(0.25)

### Purchases per Country Analysis

In [ ]:
data_country = data_original[['Country','Quantity','Total_Price']]

In [ ]:
#How many different countries do I have?
data_country['Country'].unique()

In [ ]:
len(data_country['Country'].unique())

In [ ]:
#What is the total amount of money per country?
data_country_sum = data_original[['Country','Total_Price']].groupby('Country').agg(sum)

In [ ]:
data_country_sum

In [ ]:
#Which is the country with more sales?
data_country_sum[data_country_sum['Total_Price'] == data_country_sum['Total_Price'].max()]

In [ ]:
#Which is the country with less sales?
data_country_sum[data_country_sum['Total_Price'] == data_country_sum['Total_Price'].min()]

In [ ]:
#How is the distribution of my total sales? Skewed to the left.
data_country_sum['Total_Price'].hist()

In [ ]:
#Total sales std
data_country_sum['Total_Price'].std()

In [ ]:
#Total sales mean
data_country_sum['Total_Price'].mean()

In [ ]:
#Total sales median
data_country_sum['Total_Price'].median()

In [ ]:
#Total sales q1
data_country_sum['Total_Price'].quantile(0.25)

In [ ]:
#Total sales q3
data_country_sum['Total_Price'].quantile(0.75)

In [ ]:
#Total sales iqr
data_country_sum['Total_Price'].quantile(0.75) - data_country_sum['Total_Price'].quantile(0.25)

In [ ]:
#What is the average amount of money per country?
data_country_avg = data_original[['Country','Total_Price']].groupby('Country').agg(np.mean)

In [ ]:
data_country_avg

In [ ]:
#Which is the country with more average in sales?
data_country_avg[data_country_avg['Total_Price'] == data_country_avg['Total_Price'].max()]

In [ ]:
#Which is the country with less average in sales?
data_country_avg[data_country_avg['Total_Price'] == data_country_avg['Total_Price'].min()]

In [ ]:
#How is the distribution of my average sales? Skewed to the left, but it might be possible that 
#it turn out to be a nice bell shape distribution.
data_country_avg['Total_Price'].hist()

In [ ]:
#Total sales std
data_country_avg['Total_Price'].std()

In [ ]:
#Total sales mean
data_country_avg['Total_Price'].mean()

In [ ]:
#Total sales median
data_country_avg['Total_Price'].median()

In [ ]:
#Total sales q1
data_country_avg['Total_Price'].quantile(0.25)

In [ ]:
#Total sales q3
data_country_avg['Total_Price'].quantile(0.75)

In [ ]:
#Total sales iqr
data_country_avg['Total_Price'].quantile(0.75) - data_country_avg['Total_Price'].quantile(0.25)

### Timestamp Analysis

In [ ]:
data_original['InvoiceDate'].sort_values(ascending=False)

### Product Price Analysis

In [ ]:
#What is the price of a product that people try to buy the most?
prices = data_original['UnitPrice'].value_counts()
data_original['UnitPrice'][prices.max()]

In [ ]:
#What is the price of a product that people try to buy less?
data_original['UnitPrice'][prices.min()]

### Descriptive Analytics Summary

From the descriptive analytics from above we can infer the next information:
	* The amount of times that the customer goes into the store/website and buy something is 93 in a 9 month period.
	* The average amount of money spent per customer each 9 months is 322.
	* The average amount of articles that each customer buys in a 9 month period is 199.
	* We have 38 countries in total.
	* The country with more total sales is United kingdom.
	* The country with less total sales is Saudi Arabia.
	* The country with the highest average sales is Netherlands.
	* The country with the less average sales is USA.
    * The price of products that people tend to buy the most is 2.51.
    * The price of products that people tend to buy the least is 3.39.
	* We have 2 years total of data, this means that we are able to really catch seasonallity effects and a trend. Given the conditions, I decided to use prophet to do a forecast of next 3 months for United Kingdom and Netherlands of the total amount of sales.
	* From the information above we can infer as well that United kingdom has a lot of good wholesale customers, because the mean and median are very far appart from each other. On the other hand Netherlands has a lot of direct sales because the mean and the median are close to each other. 

In [ ]:
boxplot = data_original.boxplot(column=['Quantity', 'Total_Price'])

# Apply Natural Language Processing

For this kind of task, I am going to need to implement a bag of words algorithm with a clustering technique over my descriptions to check how well my cluster fill in categories. The main idea is to identify categories over my products and been able to add more insights based on the purchases made per category.

In [ ]:
#TODO ADD NLP
#Todo try to implement a xgboost

# Time Series

In this case I am going to do a small forecast over my series with the overall sales per month. I am going to forecast the next 3 months of sales.

In [ ]:
#Todo add the time series
data_original_for = pd.read_csv('Online Retail.csv')
data_original_for['Total_Price'] = data_original['Quantity'] * data_original['UnitPrice']

In [ ]:
data_original_for

In [ ]:
type(data_original_for['InvoiceDate'][0])

In [ ]:
data_original_for['InvoiceDate'] = pd.to_datetime(data_original_for['InvoiceDate'])

In [ ]:
type(data_original_for['InvoiceDate'][0])

In [ ]:
def get_month(date):
    return date.month
data_original_for['InvoiceDate_Month'] = list(map(get_month,data_original_for['InvoiceDate']))

In [ ]:
def get_year(date):
    return date.year
data_original_for['InvoiceDate_Year'] = list(map(get_year,data_original_for['InvoiceDate']))

In [ ]:
data_original_for

In [ ]:
data_forecasting = data_original_for[['Country','Total_Price','InvoiceDate_Month','InvoiceDate_Year']].groupby(['Country','InvoiceDate_Year','InvoiceDate_Month']).agg(sum)

In [ ]:
data_forecasting

In [ ]:
united_kingdom = data_forecasting[data_forecasting.index.get_level_values(0) == 'United Kingdom']

In [ ]:
united_kingdom

In [ ]:
dates = pd.DataFrame({'year':united_kingdom.index.get_level_values(1),'month':united_kingdom.index.get_level_values(2)})

In [ ]:
dates['day'] = 1

In [ ]:
dates 

In [ ]:
dates_col = pd.to_datetime(dates)

In [ ]:
dates_col

In [ ]:
uk_for = pd.DataFrame({'dates':dates_col,'Total_Price':united_kingdom['Total_Price'].values})

In [ ]:
uk_for

In [ ]:
#I am going to fit my model up to the 18 month in such way that I am able to measure the performance.
uk_for_train = uk_for.iloc[:18,:]

In [ ]:
uk_for_train

In [ ]:
uk_for_test = uk_for.iloc[18:,:]

In [ ]:
uk_for_test